## Capstone Project - To evaluate neighborhood of England school in UK
### The objective is to compare the neighborhood of school network and find the optimal location in terms of accessibility and convenience of location for grammer schools


### A description of the problem and a discussion of the background. (15 marks)

Restaurant business to serve schools students focused in Town of Barnet. Before business opened, it is necessary to evaluate the optimal locations based on neighborhoods of campus.

### A description of the data and how it will be used to solve the problem. (15 marks)

Data required includes the UK campus geographic coordinates (latitude and longitude). Evaluate the neighborhood and type of venue categories to evaluate what kind of competition that restaurant business need to face. On the other hand, it can also evaluate the living standards by referring to categories of venues in neighborhoods.

### For the second week, the final deliverables of the project will be:

### A link to your Notebook on your Github repository, showing your code. (15 marks)
https://github.com/kitwang1/CapstoneProject/blob/main/CapstoneProjectWeek4and5.ipynb




# Data Souce : UK govt with schools in England

https://www.gov.uk/government/publications/schools-in-england

# Postal Code reference for latitude and longitude for UK postal code

https://www.freemaptools.com/download/



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!wget -q -O 'ukpostcodes.zip' https://www.freemaptools.com/download/full-postcodes/ukpostcodes.zip

In [3]:
import os
import zipfile

working_directory = '/Users/kit/CapstoneProject'

for file in os.listdir(working_directory):   # get the list of files
    if zipfile.is_zipfile(file): # if it is a zipfile, extract it
        with zipfile.ZipFile(file) as item: # treat the file as a zip
           item.extractall()  # extract it in the working directory
        
            

In [4]:
postcode = pd.read_csv('ukpostcodes.csv')

In [6]:
postcode.head(), postcode.shape

(   id  postcode   latitude  longitude
 0   1  AB10 1XG  57.144165  -2.114848
 1   2  AB10 6RN  57.137880  -2.121487
 2   3  AB10 7JB  57.124274  -2.127190
 3   4  AB11 5QN  57.142701  -2.093295
 4   5  AB11 6UL  57.137547  -2.112233,
 (1769499, 4))

In [7]:
!wget -q -O 'EduBase_Schools_July_2017.xlsx' https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/629351/EduBase_Schools_July_2017.xlsx

In [8]:
school_data = pd.read_excel('EduBase_Schools_July_2017.xlsx')

In [10]:
school_data.head()

,URN,Local authority (code),Local authority (name),Establishment number,Establishment name,Street,Locality,Address3,Town,County,Postcode,Type of establishment,Statutory highest age,Statutory lowest age,Boarders,Sixth form,UKPRN,Phase of education,Gender,Religious character,Religious ethos,Admissions policy,Website address,Telephone number,Headteacher,Establishment status,Reason establishment opened,Opening date,Parliamentary Constituency (code),Parliamentary Constituency (name),Region
0,100000,201,City of London,3614,Sir John Cass's Foundation Primary School,St James's Passage,Duke's Place,NaN,London,NaN,EC3A 5DE,Voluntary Aided School,11,3,No Boarders,Does not have a sixth form,NaN,Primary,Mixed,Church of England,Does not apply,Not applicable,www.sirjohncassprimary.org,2.072831e+09,Mr Tim Wilson,Open,Not applicable,NaT,E14000639,Cities of London and Westminster,London
1,100001,201,City of London,6005,City of London School for Girls,St Giles' Terrace,Barbican,NaN,London,NaN,EC2Y 8BB,Other Independent School,18,7,No Boarders,Has a sixth form,10013279.0,Not applicable,Girls,None,Church of England,Not collected,http://www.clsg.org.uk,2.078476e+09,Mrs Ena Harrop,Open,Not applicable,1920-01-01,E14000639,Cities of London and Westminster,London
2,100002,201,City of London,6006,St Paul's Cathedral School,2 New Change,NaN,NaN,London,NaN,EC4M 9AD,Other Independent School,13,4,Boarding School,Does not have a sixth form,10018890.0,Not applicable,Mixed,Church of England,Christian,Not collected,http://www.stpauls.co.uk/school/school.htm,2.072485e+09,Mr Neil Chippington,Open,Not applicable,1939-01-01,E14000639,Cities of London and Westminster,London
3,100003,201,City of London,6007,City of London School,Queen Victoria Street,NaN,NaN,London,NaN,EC4V 3AL,Other Independent School,18,10,No Boarders,Has a sixth form,10008165.0,Not applicable,Boys,None,None,Not collected,http://www.clsb.org/,2.074890e+09,Ms Sarah Fletcher,Open,Not applicable,1919-01-01,E14000639,Cities of London and Westminster,London
4,100005,202,Camden,1048,Thomas Coram Centre,49 Mecklenburgh Square,NaN,NaN,London,NaN,WC1N 2NY,LA Nursery School,5,3,No Boarders,Not applicable,NaN,Nursery,Mixed,Does not apply,Does not apply,Not applicable,http://www.thomascoram.camden.sch.uk/,2.075200e+09,Ms Perina Holness,Open,Not applicable,NaT,E14000750,Holborn and St Pancras,London


In [139]:
#Only retrieve col 
school_data2 = school_data[['Establishment name','Type of establishment','Town','Gender','Postcode','Region']]

In [140]:
school_data2.head()

,Establishment name,Type of establishment,Town,Gender,Postcode,Region
0,Sir John Cass's Foundation Primary School,Voluntary Aided School,London,Mixed,EC3A 5DE,London
1,City of London School for Girls,Other Independent School,London,Girls,EC2Y 8BB,London
2,St Paul's Cathedral School,Other Independent School,London,Mixed,EC4M 9AD,London
3,City of London School,Other Independent School,London,Boys,EC4V 3AL,London
4,Thomas Coram Centre,LA Nursery School,London,Mixed,WC1N 2NY,London


In [141]:
# merge postal data set to get latitude and longitude of campus
x = school_data2.merge(postcode, how='left', left_on='Postcode',right_on='postcode').drop(['id','postcode'], axis='columns')

In [142]:
x.head()

,Establishment name,Type of establishment,Town,Gender,Postcode,Region,latitude,longitude
0,Sir John Cass's Foundation Primary School,Voluntary Aided School,London,Mixed,EC3A 5DE,London,51.513662,-0.077531
1,City of London School for Girls,Other Independent School,London,Girls,EC2Y 8BB,London,51.519146,-0.094566
2,St Paul's Cathedral School,Other Independent School,London,Mixed,EC4M 9AD,London,51.513833,-0.096820
3,City of London School,Other Independent School,London,Boys,EC4V 3AL,London,51.511115,-0.099387
4,Thomas Coram Centre,LA Nursery School,London,Mixed,WC1N 2NY,London,51.525478,-0.120784


In [143]:
x.shape

(24302, 8)

In [145]:
london_set = x[x['Town']=='Barnet']

In [148]:
london_set.head()

,Establishment name,Type of establishment,Town,Gender,Postcode,Region,latitude,longitude
843,Brookhill Nursery School,LA Nursery School,Barnet,Mixed,EN4 8SD,London,51.645376,-0.165277
846,St Margaret's Nursery School,LA Nursery School,Barnet,Mixed,EN4 9NT,London,51.649405,-0.167615
854,Church Hill School,Community School,Barnet,Mixed,EN4 8NN,London,51.635273,-0.159269
858,Cromer Road Primary School,Community School,Barnet,Mixed,EN5 5HT,London,51.653674,-0.177187
863,Foulds School,Community School,Barnet,Mixed,EN5 4NR,London,51.656766,-0.211717


In [149]:
london_set.dropna(inplace=True)

/Users/kit/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [155]:
latitude, longitude = london_set.iloc[0,6:8]
latitude, longitude

(51.645375898617495, -0.16527717579869597)

In [156]:
#Visual map of London

#### Create a map of London schools based on postal codes superimposed on top.
# create map of London using latitude and longitude values
map_london= folium.Map(location=[latitude, longitude], tiles="Stamen Toner", zoom_start=10)

# add markers to map
for lat, lng, school, type in zip(london_set['latitude'], london_set['longitude'], london_set['Establishment name'], london_set['Type of establishment']):
    label = '{},{}'.format(school, type)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)
    
map_london

In [157]:
#Explore one case

LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)
url #
#  Retrieve result
results = requests.get(url).json()
results

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '6048adc3aa67e67eb8c8f679'},
 'response': {}}

In [67]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [164]:
#Repeating function to extract neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School',
                             'School Latitude', 
                             'School Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    return(nearby_venues)

In [165]:
london_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 843 to 22314
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Establishment name     24 non-null     object 
 1   Type of establishment  24 non-null     object 
 2   Town                   24 non-null     object 
 3   Gender                 24 non-null     object 
 4   Postcode               24 non-null     object 
 5   Region                 24 non-null     object 
 6   latitude               24 non-null     float64
 7   longitude              24 non-null     float64
dtypes: float64(2), object(6)
memory usage: 2.3+ KB


In [166]:
london_venue = getNearbyVenues(names=london_set['Establishment name'],
                               latitudes=london_set['latitude'],
                               longitudes=london_set['longitude'])
print(london_venue.shape)
london_venue.head()

Brookhill Nursery School
St Margaret's Nursery School
Church Hill School
Cromer Road Primary School
Foulds School
Livingstone Primary and Nursery School
Underhill School
Whitings Hill Primary School
Danegrove Primary School
Christ Church Primary School
Monken Hadley CofE Primary School
St Mary's CofE Primary School, East Barnet
Trent CofE Primary School
St Catherine's RC School
Lyonsdown School
Mount House School
Hadley Wood Primary School
Susi Earnshaw Theatre School
Queen Elizabeth's School, Barnet
East Barnet School
Queen Elizabeth's Girls' School
Grasvenor Avenue Infant School
Watling Park school
Southgate School
(115, 7)


,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brookhill Nursery School,51.645376,-0.165277,Co-op Food,51.643190,-0.164645,Grocery Store
1,Brookhill Nursery School,51.645376,-0.165277,Costa Coffee,51.642550,-0.163151,Coffee Shop
2,Brookhill Nursery School,51.645376,-0.165277,Aldi,51.644780,-0.163952,Supermarket
3,Brookhill Nursery School,51.645376,-0.165277,The Prince Of Wales,51.642136,-0.163178,Pub
4,Brookhill Nursery School,51.645376,-0.165277,ChuChinChow,51.644024,-0.162115,Chinese Restaurant


In [167]:
print(london_venue.shape)

(115, 7)


In [168]:
london_venue

,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Brookhill Nursery School,51.645376,-0.165277,Co-op Food,51.643190,-0.164645,Grocery Store
1,Brookhill Nursery School,51.645376,-0.165277,Costa Coffee,51.642550,-0.163151,Coffee Shop
2,Brookhill Nursery School,51.645376,-0.165277,Aldi,51.644780,-0.163952,Supermarket
3,Brookhill Nursery School,51.645376,-0.165277,The Prince Of Wales,51.642136,-0.163178,Pub
4,Brookhill Nursery School,51.645376,-0.165277,ChuChinChow,51.644024,-0.162115,Chinese Restaurant
5,Brookhill Nursery School,51.645376,-0.165277,Ludo's Cafe,51.642907,-0.162120,Café
6,St Margaret's Nursery School,51.649405,-0.167615,PureGym,51.649853,-0.172633,Gym / Fitness Center
7,St Margaret's Nursery School,51.649405,-0.167615,The Railway Bell (Wetherspoon),51.650485,-0.173050,Pub
8,St Margaret's Nursery School,51.649405,-0.167615,New Barnet Railway Station (NBA),51.648007,-0.172295,Train Station
9,St Margaret's Nursery School,51.649405,-0.167615,Victoria Recreation Ground,51.652064,-0.165090,Park


In [169]:
london_venue.groupby('School').count()

,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
School,,,,,,
Brookhill Nursery School,6,6,6,6,6,6
Christ Church Primary School,2,2,2,2,2,2
Church Hill School,4,4,4,4,4,4
Cromer Road Primary School,4,4,4,4,4,4
Danegrove Primary School,6,6,6,6,6,6
East Barnet School,4,4,4,4,4,4
Foulds School,2,2,2,2,2,2
Grasvenor Avenue Infant School,5,5,5,5,5,5
Hadley Wood Primary School,3,3,3,3,3,3


In [170]:
# one hot encoding
london_onehot = pd.get_dummies(london_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['School'] = london_venue['School'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Turkish Restaurant,Bakery,Bar,Bistro,Bookstore,Bus Stop,Café,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Convenience Store,Fish & Chips Shop,Flower Shop,Forest,French Restaurant,Fried Chicken Joint,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Home Service,Italian Restaurant,Locksmith,Metro Station,Modern European Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Recruiting Agency,Restaurant,Sandwich Place,School,Seafood Restaurant,Soccer Field,Soccer Stadium,Sports Club,Supermarket,Tennis Court,Theater,Train Station
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Brookhill Nursery School,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Brookhill Nursery School,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Brookhill Nursery School,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Brookhill Nursery School,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Brookhill Nursery School,0,0,0,0,0,0,0,0


In [172]:
# evaluate the frequency of venue per school

london_grouped = london_onehot.groupby('School').mean().reset_index()
london_grouped

#Let's print each neighborhood along with the top 5 most common venues


num_top_venues = 5

for school in london_grouped['School']:
    print("----"+school+"----")
    temp = london_grouped[london_grouped['School'] == school].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Brookhill Nursery School----
                venue  freq
0         Supermarket  0.17
1                Café  0.17
2  Chinese Restaurant  0.17
3         Coffee Shop  0.17
4       Grocery Store  0.17


----Christ Church Primary School----
                venue  freq
0                 Bar   0.5
1         Sports Club   0.5
2  Turkish Restaurant   0.0
3   Recruiting Agency   0.0
4       Metro Station   0.0


----Church Hill School----
                venue  freq
0                 Gym  0.25
1                Park  0.25
2                Café  0.25
3          Playground  0.25
4  Turkish Restaurant  0.00


----Cromer Road Primary School----
                venue  freq
0                 Pub  0.50
1                Park  0.25
2                Café  0.25
3  Turkish Restaurant  0.00
4          Restaurant  0.00


----Danegrove Primary School----
                venue  freq
0         Supermarket  0.17
1                Café  0.17
2  Chinese Restaurant  0.17
3         Coffee Shop  0.17
4       Grocery

In [202]:
#sorting

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['School'] = london_grouped['School']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_venues_sorted.head()


,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brookhill Nursery School,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
1,Christ Church Primary School,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
2,Church Hill School,Playground,Park,Café,Gym,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
3,Cromer Road Primary School,Pub,Park,Café,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop
4,Danegrove Primary School,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar


In [203]:
london_venues_sorted

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brookhill Nursery School,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
1,Christ Church Primary School,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
2,Church Hill School,Playground,Park,Café,Gym,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
3,Cromer Road Primary School,Pub,Park,Café,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop
4,Danegrove Primary School,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
5,East Barnet School,Home Service,Locksmith,Modern European Restaurant,Forest,Convenience Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Flower Shop,Fish & Chips Shop
6,Foulds School,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
7,Grasvenor Avenue Infant School,Recruiting Agency,Soccer Field,Café,Gym,Coffee Shop,Convenience Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
8,Hadley Wood Primary School,Train Station,Soccer Field,Convenience Store,Construction & Landscaping,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
9,Livingstone Primary and Nursery School,Construction & Landscaping,Park,Flower Shop,Gym / Fitness Center,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest


In [204]:
#Run k-means to cluster the neighborhood into 3 clusters.
# set number of clusters
kclusters = 2

london_grouped_clustering = london_grouped.drop('School', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

In [205]:
london_set.head

<bound method NDFrame.head of                                Establishment name     Type of establishment  \
843                      Brookhill Nursery School         LA Nursery School   
846                  St Margaret's Nursery School         LA Nursery School   
854                            Church Hill School          Community School   
858                    Cromer Road Primary School          Community School   
863                                 Foulds School          Community School   
870        Livingstone Primary and Nursery School          Community School   
878                              Underhill School          Community School   
879                  Whitings Hill Primary School          Community School   
884                      Danegrove Primary School          Community School   
886                  Christ Church Primary School    Voluntary Aided School   
888             Monken Hadley CofE Primary School    Voluntary Aided School   
892    St Mary's CofE 

In [206]:
london_venues_sorted.head

<bound method NDFrame.head of                                         School       1st Most Common Venue  \
0                     Brookhill Nursery School                 Coffee Shop   
1                 Christ Church Primary School                         Bar   
2                           Church Hill School                  Playground   
3                   Cromer Road Primary School                         Pub   
4                     Danegrove Primary School                 Coffee Shop   
5                           East Barnet School                Home Service   
6                                Foulds School                         Bar   
7               Grasvenor Avenue Infant School           Recruiting Agency   
8                   Hadley Wood Primary School               Train Station   
9       Livingstone Primary and Nursery School  Construction & Landscaping   
10                            Lyonsdown School               Train Station   
11             Queen Elizabeth's G

In [207]:
# add clustering labels
london_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_set

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_venues_sorted.set_index('School'), on='Establishment name')

london_merged.head() # check the last columns!

,Establishment name,Type of establishment,Town,Gender,Postcode,Region,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
843,Brookhill Nursery School,LA Nursery School,Barnet,Mixed,EN4 8SD,London,51.645376,-0.165277,1.0,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
846,St Margaret's Nursery School,LA Nursery School,Barnet,Mixed,EN4 9NT,London,51.649405,-0.167615,1.0,Train Station,Park,Pub,Gym / Fitness Center,Fish & Chips Shop,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
854,Church Hill School,Community School,Barnet,Mixed,EN4 8NN,London,51.635273,-0.159269,1.0,Playground,Park,Café,Gym,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
858,Cromer Road Primary School,Community School,Barnet,Mixed,EN5 5HT,London,51.653674,-0.177187,1.0,Pub,Park,Café,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop
863,Foulds School,Community School,Barnet,Mixed,EN5 4NR,London,51.656766,-0.211717,0.0,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop


In [209]:
london_merged.dropna(inplace = True)

In [213]:
london_merged

,Establishment name,Type of establishment,Town,Gender,Postcode,Region,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
843,Brookhill Nursery School,LA Nursery School,Barnet,Mixed,EN4 8SD,London,51.645376,-0.165277,1.0,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
846,St Margaret's Nursery School,LA Nursery School,Barnet,Mixed,EN4 9NT,London,51.649405,-0.167615,1.0,Train Station,Park,Pub,Gym / Fitness Center,Fish & Chips Shop,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
854,Church Hill School,Community School,Barnet,Mixed,EN4 8NN,London,51.635273,-0.159269,1.0,Playground,Park,Café,Gym,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
858,Cromer Road Primary School,Community School,Barnet,Mixed,EN5 5HT,London,51.653674,-0.177187,1.0,Pub,Park,Café,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop
863,Foulds School,Community School,Barnet,Mixed,EN5 4NR,London,51.656766,-0.211717,0.0,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
870,Livingstone Primary and Nursery School,Community School,Barnet,Mixed,EN4 9BU,London,51.654560,-0.167205,1.0,Construction & Landscaping,Park,Flower Shop,Gym / Fitness Center,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
878,Underhill School,Community School,Barnet,Mixed,EN5 2LZ,London,51.645000,-0.207223,1.0,Grocery Store,Train Station,Gym / Fitness Center,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop,Convenience Store
879,Whitings Hill Primary School,Community School,Barnet,Mixed,EN5 2QY,London,51.646173,-0.216991,1.0,Seafood Restaurant,Café,Construction & Landscaping,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
884,Danegrove Primary School,Community School,Barnet,Mixed,EN4 8UD,London,51.643610,-0.157616,1.0,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
886,Christ Church Primary School,Voluntary Aided School,Barnet,Mixed,EN5 4NS,London,51.657672,-0.213315,0.0,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop


In [222]:
#Visualize map

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [230]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['latitude'], london_merged['longitude'], london_merged['Establishment name'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

### Examine Cluster

##### examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.


In [234]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Type of establishment,Region,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
863,Community School,London,51.656766,-0.211717,0.0,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
886,Voluntary Aided School,London,51.657672,-0.213315,0.0,Bar,Sports Club,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
16592,Academy Converter,London,51.655857,-0.213358,0.0,Bar,Sports Club,Bus Stop,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest


In [235]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Type of establishment,Region,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
843,LA Nursery School,London,51.645376,-0.165277,1.0,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
846,LA Nursery School,London,51.649405,-0.167615,1.0,Train Station,Park,Pub,Gym / Fitness Center,Fish & Chips Shop,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
854,Community School,London,51.635273,-0.159269,1.0,Playground,Park,Café,Gym,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop
858,Community School,London,51.653674,-0.177187,1.0,Pub,Park,Café,Train Station,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop
870,Community School,London,51.654560,-0.167205,1.0,Construction & Landscaping,Park,Flower Shop,Gym / Fitness Center,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
878,Community School,London,51.645000,-0.207223,1.0,Grocery Store,Train Station,Gym / Fitness Center,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest,Flower Shop,Fish & Chips Shop,Convenience Store
879,Community School,London,51.646173,-0.216991,1.0,Seafood Restaurant,Café,Construction & Landscaping,Train Station,Convenience Store,Grocery Store,Greek Restaurant,Fried Chicken Joint,French Restaurant,Forest
884,Community School,London,51.643610,-0.157616,1.0,Coffee Shop,Café,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bistro,Bookstore,Bus Stop,Bar
892,Voluntary Aided School,London,51.641018,-0.161493,1.0,Coffee Shop,Café,Greek Restaurant,Fried Chicken Joint,Pub,Chinese Restaurant,Grocery Store,Supermarket,Bus Stop,Bookstore
896,Voluntary Aided School,London,51.650307,-0.152849,1.0,Turkish Restaurant,Greek Restaurant,Fish & Chips Shop,Bakery,Italian Restaurant,Bistro,Grocery Store,Café,Fried Chicken Joint,French Restaurant


# Result of analysis

## Based on above analysis, it can be found that Cluster 0 has much less restaurants and Cluster 1 has much more restaurants. It means if restaurant business is setup at Cluster 1, it can definitely face less competition and most of school students would be served by this restaurant business. Due to proximity of schools, it is more competitive in terms of ease to access the restaurant and transport lead time is limited.

## Meanwhile, it is also found that Cluster 1 is more widely spread over the Town of Barnet and it means that it is hard to serve most of schools in the town as there's no centroid that allow ease of access without close substitute / competitions. 

